In [ ]:
pip install pandas

In [ ]:
import os
import pandas as pd

In [ ]:
pip install pdfplumber

In [ ]:
import os
import pdfplumber

In [ ]:
# Annual report Transfer
# Folder Path
filepath = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\annual report'

# Retrieve all file names in the folder
filename = os.listdir(filepath)

# Creat a pathlist to store the complete path of each file
pathlist = []

# After adding each file name to the folder path, form a complete path
for eachname in filename:
    pathlist.append(os.path.join(filepath, eachname))

# Traverse each file path
for eachfile in pathlist:
    # replace"Annual reports" to "transfer result"and change the format from pdf to txt
    txtpath = eachfile.replace("annual report", "transfer result").replace("pdf", "txt")
    
    # Ensure that the target folder exists
    os.makedirs(os.path.dirname(txtpath), exist_ok=True)
    
    try:
        # Create and open a txt file in write mode
        with open(txtpath, "w", encoding='utf-8') as txt:
            # use pdfplumber to PDF files
            with pdfplumber.open(eachfile) as pdf:
                for page in pdf.pages:
                    # extract word and write it in txt file
                    text = page.extract_text()
                    if text:  # Write in only when the text is not empty
                        txt.write(text)
        print(f"Extracted text saved to {txtpath}")
    except Exception as e:
        print(f"Failed to process {eachfile}: {e}")



In [ ]:
pip install openpyxl

In [ ]:
 # English word frequency statistics
import os
import re
import pandas as pd

# Ensure that the necessary libraries are installed
try:
    import openpyxl
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
    import openpyxl

# Define folder path and AI related vocabulary list
folder_path = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\transfer result'
ai_terms = [
    "Artificial Intelligence", "Big Data", "Cloud Computing", "cloud storage",
    "Fintech", "Digital", "Technology", "Technical", "Innovation", "Tech Firm",
    "Machine Learning", "Blockchain Technology", "Information System", "Internet of Things","Iot",
    "Digital Payments", "Data-Driven", "Green Technology", "Cybersecurity",
    "Risk-based Assessment System", "Supervisory Rating System", "Risk Management", "Network Security",
    "Credit Scoring", "Fraud Detection", "Data Privacy Protection", "Online Insurance",
    "Crowdfunding", "Online Lending", "Online Investment", "Algorithmic Trading",
    "Neural Networks", "Data Mining", "Internet Finance", "Electronic Money", "AI Customer Service"
]

# Define a list of stop words
stop_words = {
    "i", "we", "are", "is", "the", "and", "to", "of", "in", "a", "that", "it", "with", "as", "for",
    "was", "on", "by", "at", "an", "be", "this", "which", "or", "from", "but", "not", "have", "had",
    "were", "has", "been", "they", "you", "he", "she", "his", "her", "their", "them", "my", "your",
    "its", "our", "us", "me", "do", "does", "did", "can", "could", "will", "would", "shall", "should",
    "may", "might", "must", "am", "were", "being", "having", "than", "because", "about", "which"
}

# Dictionary for storing results
results = []

# Attempt to read files using multiple encodings
def read_file_with_multiple_encodings(file_path):
    encodings = ['utf-8', 'ISO-8859-1', 'ANSI', 'latin1']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"None of the encodings worked for {file_path}")

# Traverse every TXT file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        try:
            text = read_file_with_multiple_encodings(file_path)
        except UnicodeDecodeError as e:
            print(f"Failed to read {file_path}: {e}")
            continue

        # Tokenize the text into words and filter out stop words
        words = re.findall(r'\b\w+\b', text)
        meaningful_words = [word for word in words if word.lower() not in stop_words]

        # Calculate the total number of meaningful words in the file
        total_meaningful_words = len(meaningful_words)

        # Count the frequency of each AI related vocabulary
        word_freq = {}
        for term in ai_terms:
            # Use re.findall to calculate the number of occurrences of each word (ignoring capitalization)
            word_freq[term] = len(re.findall(re.escape(term), text, re.IGNORECASE))

        # Add the results to the list
        result = {'File Name': filename}
        result.update(word_freq)
        result['Total Meaningful Words'] = total_meaningful_words
        results.append(result)

# Transfer the results to DataFrame
df = pd.DataFrame(results)

# Save the results to Excel files
output_file = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\transfer result\ai_terms_frequency.xlsx'
df.to_excel(output_file, index=False)

print(f"Word frequency analysis saved to {output_file}")


In [ ]:
pip install tika

In [ ]:

import re

def extract_text_between_markers(file_path, start_marker, end_marker, output_file_path):
    # Attempt to read the file with multiple encodings
    encodings = ['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                text = file.read()
            break  # Exit the loop if reading is successful
        except UnicodeDecodeError:
            print(f"Failed to read {file_path} with {encoding} encoding.")
            continue
    else:
        print(f"Unable to read {file_path} with any of the attempted encodings.")
        return

    # Using regular expressions for matching
    start_pattern = re.compile(re.escape(start_marker), re.IGNORECASE)
    end_pattern = re.compile(re.escape(end_marker), re.IGNORECASE)
    
    start_match = start_pattern.search(text)
    end_match = end_pattern.search(text)
    
    if not start_match or not end_match:
        print(f"Unable to find markers in {file_path}.")
        return
    
    start_index = start_match.end()
    end_index = end_match.start()
    
    # Extract text between identifiers
    extracted_text = text[start_index:end_index].strip()
    
    # Save the extracted text to a new file
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(extracted_text)
    
    print(f"Extracted text saved to {output_file_path}")

# Example usage
input_file_path = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\transfer result\United States_US3167731005_2020.txt'
start_marker = 'Management’s Discussion'
end_marker = 'CONSOLIDATED BALANCE SHEETS'
output_file_path = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\transfer result\United States_US3167731005_2020(1).txt'

extract_text_between_markers(input_file_path, start_marker, end_marker, output_file_path)



In [ ]:
import os
import pdfplumber
import re

def extract_text_between_markers_from_pdf(pdf_path, start_marker, end_marker, output_file_path):
    # Read PDF file content
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_text += page.extract_text()
    
    # Using regular expressions for matching
    start_pattern = re.compile(re.escape(start_marker), re.IGNORECASE)
    end_pattern = re.compile(re.escape(end_marker), re.IGNORECASE)
    
    start_match = start_pattern.search(extracted_text)
    end_match = end_pattern.search(extracted_text)
    
    if not start_match or not end_match:
        print(f"Unable to find markers in {pdf_path}.")
        return
    
    start_index = start_match.end()
    end_index = end_match.start()
    
    # Check if the sequence of identifiers is correct
    if start_index >= end_index:
        print(f"Start marker appears after end marker or markers are not found in the correct order in {pdf_path}.")
        return
    
    # Extract text between identifiers
    text_between_markers = extracted_text[start_index:end_index].strip()
    
    # Save the extracted text to a new file
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(text_between_markers)
    
    print(f"Extracted text saved to {output_file_path}")

def process_all_pdfs_in_folder(folder_path, start_marker, end_marker, output_folder_path):
    # Traverse all PDF files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            output_file_path = os.path.join(output_folder_path, f"extracted_{filename[:-4]}.txt")
            extract_text_between_markers_from_pdf(pdf_path, start_marker, end_marker, output_file_path)

# examples
folder_path = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\annual report'
output_folder_path = r'C:\Users\Trist\Desktop\Tohoku University\tohoku_bootcamp-main\Stata camp\Textual Analysis\transfer result(2)'
start_marker = 'Management’s Discussion'
end_marker = 'CONSOLIDATED BALANCE SHEETS'

# Create output folder (if it doesn't exist)
os.makedirs(output_folder_path, exist_ok=True)

process_all_pdfs_in_folder(folder_path, start_marker, end_marker, output_folder_path)
